This notebook analyses the results of running the RMA tool

In [ ]:
import sys
if "../" not in sys.path:
    sys.path.append("../")

import os
import glob
import re

import matplotlib.pyplot as plt
import pandas as pd

### Check results

In [ ]:
result_csvs_restr = glob.glob("../data/processed/restructure_test/*.csv")
result_csvs_restr

In [ ]:
processed_count_restr = {}
for csv in result_csvs_restr:
    df = pd.read_csv(csv, index_col=0, encoding="utf8")
    processed_count_restr[csv] = df.shape

In [ ]:
proc_iams_total = 0
proc_aleph_total = 0
for k,v in processed_count_restr.items():
    if "v2" in k:
        proc_iams_total += v[0]
    else:
        proc_aleph_total += v[0] 

In [ ]:
proc_iams_total, proc_aleph_total

### Check results

In [ ]:
result_csvs_vec = glob.glob("../data/processed/vectorised/*.csv")
result_csvs_vec

In [ ]:
processed_count_vec = {}
for csv in result_csvs_vec:
    df = pd.read_csv(csv, index_col=0, encoding="utf8")
    processed_count_vec[csv] = df.shape

In [ ]:
proc_iams_total = 0
proc_aleph_total = 0
for k,v in processed_count_vec.items():
    if "v2" in k:
        proc_iams_total += v[0]
    else:
        proc_aleph_total += v[0] 

In [ ]:
processed_count_vec

In [ ]:
for (k_vec, v_vec), (k_rest, v_rest) in zip(processed_count_vec.items(), processed_count_restr.items()):
    if v_vec[0] != v_rest[0]:
        print(k_vec)

In [ ]:
processed_count_vec["../data/processed/vectorised\Western Manuscripts_v2_matches.csv"], processed_count_restr["../data/processed/restructure_test\Western Manuscripts_v2_matches.csv"]

In [ ]:
proc_iams_total, proc_aleph_total

In [ ]:
vec_df = pd.read_csv("../data/processed/vectorised\Western Manuscripts_v2_matches.csv", index_col=0, encoding="utf8")
rest_df = pd.read_csv("../data/processed/restructure_test\Western Manuscripts_v2_matches.csv", index_col=0, encoding="utf8")

In [ ]:
rest_df.loc["Add MS 89261/1/1026"]["Term"].values

In [ ]:
rest_df

In [ ]:
vec_df

In [ ]:
vec_df.index.unique()

In [ ]:
vec_df.index.difference(rest_df.index)

### Combine 1800s books

In [ ]:
pre1800 = [
    '../data/processed\\epBooks1800s_1_matches.csv',
    '../data/processed\\epBooks1800s_2_matches.csv',
    '../data/processed\\epBooks1800s_3_matches.csv',
    '../data/processed\\epBooks1800s_4_matches.csv'
]

In [ ]:
pre1800_dfs = [pd.read_csv(match, index_col=0, encoding="utf8") for match in pre1800]
pre1800_df = pd.concat(pre1800_dfs)

In [ ]:
# pre1800_df.to_csv('../data/processed\\epBooks1800s_matches.csv', encoding="utf8")
# pre1800_df.to_excel('../data/processed\\epBooks1800s_matches.xlsx')

### Total records check

In [ ]:
total_rows = 0
for csv in csvs:
    if "utf8" in csv:
        continue
    elif "v2" not in csv:
        continue
    print(csv)
    df = pd.read_csv(csv, encoding="utf8", nrows=5)
    print(not ({"Title", "Scope and content", "Date range", "Reference"} - set(df.columns)))

Combined 6m rows, 3.5m Aleph, 2.5 IAMS

### Profiling vectorised steps

In [ ]:
df = pd.read_csv("../data/interim/Maps.csv")

%%timeit 
df["Title [245]"].str.contains("arabesque", regex=False, na=False, case=False)

%%timeit 
df["Title [245]"].str.contains("arabesque", regex=True, na=False, case=False)

%%timeit 
df["Title [245]"].str.split("arab", regex=True, n=1, expand=True)

%%timeit
df.iloc[:50]["Title [245]"].str.split("arab", regex=True, n=1, expand=True)

So can do 2 contains for less time than 1 regex search. If can reduce each term to 2 lemmas then use regex on version of that then will be quicker. Fine for plurals. Probably have a third column for regexes and a fourth column for whether to do regex immediately - in which case should just split.